In [1]:
import os
import time
import pandas as pd
import numpy as np
import datetime
import pymysql
from sqlalchemy import create_engine
import FinanceDataReader as fdr
from tqdm import tqdm

In [5]:
code_data = pd.read_csv('./raw_data.csv', index_col = 0)
code_data['상장일'] = pd.to_datetime(code_data['상장일'])
code_data['종목코드'] = code_data['종목코드'].apply(lambda x: str(x).zfill(6))

In [6]:
code_data[:5]

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DL,000210,기타 금융업,지주회사,1976-02-02,12월,전병욱,http://www.dlholdings.co.kr,서울특별시
1,DRB동일,004840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
2,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
3,GS,078930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
4,GS글로벌,001250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시


# 과제1

In [7]:
lst_stock = code_data.values.tolist()

In [8]:

dic_code2company = {}
for row in lst_stock:
    company, code, date = row[0], row[1], row[4]
    if date <= datetime.datetime(2018, 1, 2):
        dic_code2company[code] = company

In [9]:
OF = open('assignment1_sql.txt', 'w', encoding = 'utf-8')
for code in dic_code2company.keys():
    data = '{}\t{}\n'.format(code, dic_code2company[code])
    OF.write(data)
OF.close()

# 과제2

In [2]:
db_dsml = pymysql.connect(
    host = 'localhost', 
    port = 3306, 
    user = 'stock_user', 
    passwd = 'bigdata', 
    db = 'stock', 
    charset = 'utf8'
)

cursor = db_dsml.cursor()

In [298]:
dic_code2date = {}

OF = open('assignment2_백억.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2company.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2018-01-01' AND '2021-12-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for row in lst_stock:
        if row[4] * row[5] >= 10000000000:
            data = '{}\t{}\n'.format(code, row[0].strftime('%Y%m%d'))
            OF.write(data)
            if code not in dic_code2date.keys():
                dic_code2date[code] = []
                dic_code2date[code].append(row[0])
            else:
                dic_code2date[code].append(row[0])
OF.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1999/1999 [00:31<00:00, 63.14it/s]


# 과제3

In [22]:
OF = open('assignment3_백억.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2date.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2018-01-01' AND '2020-12-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for i, row_lst_stock in enumerate(lst_stock):
        # 예외 처리
        if (i < 9) or (i >= len(lst_stock)-1):
            continue  
        date = row_lst_stock[0]
        if date not in dic_code2date[code]:
            continue
        
        # 11 days data
        sub_stock = lst_stock[i-9:i+1]
        lst_data = []
        for row_sub_stock in sub_stock:
            open, high, low, close, volume = row_sub_stock[1:6]
            trading_value = close * volume
            lst_data += [open, high, low, close, trading_value]
            del open
        data = ','.join(map(str, lst_data))
        
        # label
        label = int(lst_stock[i+1][-1] >= 0.03)
        
        result = '{}\t{}\t{}\t{}\n'.format(code, date.strftime("%Y%m%d"), data, label)
        OF.write(result)
OF.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1878/1878 [00:28<00:00, 66.82it/s]


# 과제 3-2 (상반기)

In [299]:
OF = open('assignment3-2_백억.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2date.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2021-01-01' AND '2021-06-30'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for i, row_lst_stock in enumerate(lst_stock):
        # 예외 처리
        if (i < 9) or (i >= len(lst_stock)-1):
            continue  
        date = row_lst_stock[0]
        if date not in dic_code2date[code]:
            continue
        
        # 11 days data
        sub_stock = lst_stock[i-9:i+1]
        lst_data = []
        for row_sub_stock in sub_stock:
            open, high, low, close, volume = row_sub_stock[1:6]
            trading_value = close * volume
            lst_data += [open, high, low, close, trading_value]
            del open
        data = ','.join(map(str, lst_data))
        
        # label
        label = int(lst_stock[i+1][-1] >= 0.03)
        
        result = '{}\t{}\t{}\t{}\n'.format(code, date.strftime("%Y%m%d"), data, label)
        OF.write(result)
OF.close()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1878/1878 [00:07<00:00, 258.32it/s]


# 과제 3-3 (하반기)

In [300]:
OF = open('assignment3-3_백억.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2date.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2021-07-01' AND '2021-12-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for i, row_lst_stock in enumerate(lst_stock):
        # 예외 처리
        if (i < 9) or (i >= len(lst_stock)-1):
            continue  
        date = row_lst_stock[0]
        if date not in dic_code2date[code]:
            continue
        
        # 11 days data
        sub_stock = lst_stock[i-9:i+1]
        lst_data = []
        for row_sub_stock in sub_stock:
            open, high, low, close, volume = row_sub_stock[1:6]
            trading_value = close * volume
            lst_data += [open, high, low, close, trading_value]
            del open
        data = ','.join(map(str, lst_data))
        
        # label
        label = int(lst_stock[i+1][-1] >= 0.03)
        
        result = '{}\t{}\t{}\t{}\n'.format(code, date.strftime("%Y%m%d"), data, label)
        OF.write(result)
OF.close()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1878/1878 [00:07<00:00, 251.66it/s]


# 과제 4-1
학습 데이터셋(assignment3.txt)를 받아서 모델을 학습하여 저장(model_????.pickle)

In [3]:
import numpy as np
import sklearn.metrics as metrics
import pickle
from sklearn.linear_model import LogisticRegression

IF=open("assignment3_백억.txt",'r')
lst_code_date=[]
trainX=[]
trainY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    trainX.append(list(map(int, x.split(","))))
    trainY.append(int(y))
trainX=np.array(trainX)
trainY=np.array(trainY)

clf = LogisticRegression(C=100,solver='lbfgs',max_iter=1000,n_jobs=1)
clf.fit(trainX, trainY)

with open('model_logistic.pickle', 'wb') as f:
    pickle.dump(clf, f)

# 과제 4-2 (상반기)

학습한 모델(model_????.pickle)과 시험 데이터셋(assignment3-2.txt)을 받아서 상승 여부 예측

In [4]:
import numpy as np
import sklearn.metrics as metrics
import pickle
from sklearn.linear_model import LogisticRegression

IF=open("assignment3-2_백억.txt",'r')
lst_code_date=[]
testX=[]
testY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    testX.append(list(map(int, x.split(","))))
    testY.append(int(y))
testX=np.array(testX)
testY=np.array(testY)

with open('model_logistic.pickle', 'rb') as f:
    clf = pickle.load(f)
    
predY = clf.predict_proba(testX) # predict_proba 함수는 예측한 값을 확률 값으로 출력
predY2 = clf.predict(testX)  # predict 함수는 예측한 값을 이진 값(1 또는 0)으로 출력


# 과제 4-3 (상반기)

예측한 값으로부터 주문 요청 일지(assignment4.txt) 작성

In [5]:
lst_output=[]
for (code, date), y in zip(lst_code_date, predY):
    if y[1] >=0.47 : # 해당일의 다음날 2% 이상 상승될 확률이 0.35이상이면
        lst_output.append([code, date, "buy", "r90"])  # 해당일에 보유 금액의 10%를 매수 요청
        lst_output.append([code, date+'n', "sell", "all"])  # 다음날(다음날이라는 의미로 날짜 뒤에 n을 추가) 전부 매도 요청
        
lst_output.sort(key=lambda x:x[1]) # date 기준으로 주문 요청 결과를 정렬

OF=open("trading2022firsthalf.txt",'w') # 주문 요청 일지를 파일로 기록
for row in lst_output:
    OF.write("\t".join(map(str, row))+"\n")
OF.close()

# 상반기 수익률



In [6]:
start_money = 10000000 # 초기 현금 1천만원
money = start_money
dic_code2num ={}  # 보유 종목

IF=open("trading2022firsthalf.txt",'r')


for i, line in enumerate(tqdm(IF)): #주문 일지를 한 줄 읽어 옴
    code, date, request, amount = line.strip().split("\t")
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                between  '2021-01-01' AND '2021-08-01'
                '''.format(code,date)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock1 = stock.values.tolist()
        
    
    if 'n' in date:
        for ii, row in enumerate(lst_stock1):
            date2 = row[0].strftime("%Y%m%d")
            if date2== date.rstrip('n'):
                nextday_close=lst_stock1[ii+1][4]
            
    else:
        for row2 in lst_stock1:
            today_date=row2[0].strftime("%Y%m%d")
            if today_date in date:
                today_close= row2[4]

                
                
    if request == 'buy': # buy인 경우
        if amount.startswith('r'):
            request_money = money * float(amount.lstrip("r")) / 100
        elif amount == 'all':
            request_money = money
        elif amount.isdigit():
            request_money = int(amount)
            
        # elif amount == ~~~~~    ##### 기타 필요한 매수 요청 옵션이 있을 시 작성
        else:
            raise Exception('Not permitted option')
        request_money = min(request_money, money)
        buy_num = int(request_money / today_close)
        money -= buy_num * today_close  # 현재 금액(money)을 실제 매수액을 뺀 만큼 업데이트
        if code not in dic_code2num:
            dic_code2num[code] = 0
        dic_code2num[code] += buy_num # 보유 종목 데이터에 구매 종목(code)를 매수 개수 만큼 증가
    if request == 'sell': # sell인 경우
        if amount == 'all':
            sell_num = dic_code2num[code]
        # elif amount == ~~~~~    ##### 기타 필요한 매도 요청 옵션이 있을 시 작성
        else:
            raise Exception('Not permitted option')            
        money += sell_num * nextday_close
        dic_code2num[code] -= sell_num
        if dic_code2num[code] == 0:
            del dic_code2num[code]
IF.close()            
            
if dic_code2num != {}: # 매매가 종료되었는데 보유 종목이 있으면
    raise Exception('Not empty stock') 

print("Final earning rate : {:.2f} %".format(float((money-start_money) / start_money * 100)))

1220it [00:04, 296.06it/s]

Final earning rate : 203.17 %


# 하반기 

In [305]:
# 훈련데이터

import numpy as np
import sklearn.metrics as metrics
import pickle
from sklearn.linear_model import LogisticRegression

IF=open("assignment3_백억.txt",'r')
lst_code_date=[]
trainX=[]
trainY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    trainX.append(list(map(int, x.split(","))))
    trainY.append(int(y))
trainX=np.array(trainX)
trainY=np.array(trainY)

clf = LogisticRegression(C=100,solver='lbfgs',max_iter=1000,n_jobs=1)
clf.fit(trainX, trainY)

with open('model_logistic.pickle', 'wb') as f:
    pickle.dump(clf, f)

# 과제 4-3 (하반기)

In [7]:
import numpy as np
import sklearn.metrics as metrics
import pickle
from sklearn.linear_model import LogisticRegression

IF=open("assignment3-3_백억.txt",'r')
lst_code_date=[]
testX=[]
testY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    testX.append(list(map(int, x.split(","))))
    testY.append(int(y))
testX=np.array(testX)
testY=np.array(testY)

with open('model_logistic.pickle', 'rb') as f:
    clf = pickle.load(f)
    
predY = clf.predict_proba(testX) # predict_proba 함수는 예측한 값을 확률 값으로 출력
predY2 = clf.predict(testX)  # predict 함수는 예측한 값을 이진 값(1 또는 0)으로 출력


# 4-4 (하반기)

In [8]:
lst_output=[]
for (code, date), y in zip(lst_code_date, predY):
    if y[1] >=0.4795 : # 해당일의 다음날 2% 이상 상승될 확률이 0.35이상이면
        lst_output.append([code, date, "buy", "r90"])  # 해당일에 보유 금액의 10%를 매수 요청
        lst_output.append([code, date+'n', "sell", "all"])  # 다음날(다음날이라는 의미로 날짜 뒤에 n을 추가) 전부 매도 요청
        
lst_output.sort(key=lambda x:x[1]) # date 기준으로 주문 요청 결과를 정렬

OF=open("trading2022secondhalf.txt",'w') # 주문 요청 일지를 파일로 기록
for row in lst_output:
    OF.write("\t".join(map(str, row))+"\n")
OF.close()

# 하반기 수익률

In [9]:
start_money = 10000000 # 초기 현금 1천만원
money = start_money
dic_code2num ={}  # 보유 종목

IF=open("trading2022secondhalf.txt",'r')


for i, line in enumerate(tqdm(IF)): #주문 일지를 한 줄 읽어 옴
    code, date, request, amount = line.strip().split("\t")
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                between  '2021-01-01' AND '2022-01-01'
                '''.format(code,date)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock1 = stock.values.tolist()
        
    
    if 'n' in date:
        for ii, row in enumerate(lst_stock1):
            date2 = row[0].strftime("%Y%m%d")
            if date2== date.rstrip('n'):
                nextday_close=lst_stock1[ii+1][4]
            
    else:
        for row2 in lst_stock1:
            today_date=row2[0].strftime("%Y%m%d")
            if today_date in date:
                today_close= row2[4]

                
                
    if request == 'buy': # buy인 경우
        if amount.startswith('r'):
            request_money = money * float(amount.lstrip("r")) / 100
        elif amount == 'all':
            request_money = money
        elif amount.isdigit():
            request_money = int(amount)
            
        # elif amount == ~~~~~    ##### 기타 필요한 매수 요청 옵션이 있을 시 작성
        else:
            raise Exception('Not permitted option')
        request_money = min(request_money, money)
        buy_num = int(request_money / today_close)
        money -= buy_num * today_close  # 현재 금액(money)을 실제 매수액을 뺀 만큼 업데이트
        if code not in dic_code2num:
            dic_code2num[code] = 0
        dic_code2num[code] += buy_num # 보유 종목 데이터에 구매 종목(code)를 매수 개수 만큼 증가
    if request == 'sell': # sell인 경우
        if amount == 'all':
            sell_num = dic_code2num[code]
        # elif amount == ~~~~~    ##### 기타 필요한 매도 요청 옵션이 있을 시 작성
        else:
            raise Exception('Not permitted option')            
        money += sell_num * nextday_close
        dic_code2num[code] -= sell_num
        if dic_code2num[code] == 0:
            del dic_code2num[code]
IF.close()            
            
if dic_code2num != {}: # 매매가 종료되었는데 보유 종목이 있으면
    raise Exception('Not empty stock') 

print("Final earning rate : {:.2f} %".format(float((money-start_money) / start_money * 100)))

124it [00:00, 194.72it/s]

Final earning rate : 39.95 %


8.391740913114939